In [1]:
import os
import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
from random import shuffle
#import pickle, datetime
import math

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras import optimizers
from keras.preprocessing import sequence
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import  PIL.Image

Using TensorFlow backend.


## Data Loading

In [2]:
N_CATEGORY = 36
BATCH_SIZE = 128
train_data_dir = '/Users/jdu12/Desktop/research/input'

train_datagen = ImageDataGenerator(
    #rescale=1. / 255,
    horizontal_flip=False)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(227, 227),
    batch_size=BATCH_SIZE,
    class_mode='categorical')
#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, N_CATEGORY)

Found 5075 images belonging to 36 classes.


## AlexNet Architecture 

### Model Parameters 

In [3]:
DROPOUT = 0.5
model_input = Input(shape = (227, 227, 3))

# First convolutional Layer (96x11x11)
z = Conv2D(filters = 96, kernel_size = (11,11), strides = (4,4), activation = "relu")(model_input)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Second convolutional Layer (256x5x5)
z = ZeroPadding2D(padding = (2,2))(z)
z = Convolution2D(filters = 256, kernel_size = (5,5), strides = (1,1), activation = "relu")(z)
z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = BatchNormalization()(z)

# Rest 3 convolutional layers
z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 384, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = ZeroPadding2D(padding = (1,1))(z)
z = Convolution2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation = "relu")(z)

z = MaxPooling2D(pool_size = (3,3), strides=(2,2))(z)
z = Flatten()(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

z = Dense(4096, activation="relu")(z)
z = Dropout(DROPOUT)(z)

final_dim = 1 if N_CATEGORY == 2 else N_CATEGORY
final_act = "sigmoid" if N_CATEGORY == 2 else "softmax"
model_output = Dense(final_dim, activation=final_act)(z)

model = Model(model_input, model_output)
#model.summary()

In [4]:
'''
import time
import pylab as pl
from IPython import display
pl.style.use('ggplot')
%matplotlib inline

class Histories(keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss = []
        self.val_loss = []
        self.val_acc = []

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        self.acc.append(logs['acc'])
        self.loss.append(logs['loss'])
        self.val_acc.append(logs['val_acc'])
        self.val_loss.append(logs['val_loss'])
        
        pl.hold(True)
        plt.rcParams["figure.figsize"] = (8,5)
        pl.plot(self.acc, 'r')
        pl.plot(self.loss, 'b')
        pl.plot(self.val_acc, 'g')
        pl.plot(self.val_loss, 'k')
        pl.legend(['Train acc','Train loss','Valid acc', 'Valid loss'], loc=2)
        display.clear_output(wait=True)
        display.display(pl.gcf())
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return
'''

'\nimport time\nimport pylab as pl\nfrom IPython import display\npl.style.use(\'ggplot\')\n%matplotlib inline\n\nclass Histories(keras.callbacks.Callback):\n\n    def on_train_begin(self, logs={}):\n        self.acc = []\n        self.loss = []\n        self.val_loss = []\n        self.val_acc = []\n\n    def on_train_end(self, logs={}):\n        return\n\n    def on_epoch_begin(self, epoch, logs={}):\n        return\n\n    def on_epoch_end(self, epoch, logs={}):\n        self.acc.append(logs[\'acc\'])\n        self.loss.append(logs[\'loss\'])\n        self.val_acc.append(logs[\'val_acc\'])\n        self.val_loss.append(logs[\'val_loss\'])\n        \n        pl.hold(True)\n        plt.rcParams["figure.figsize"] = (8,5)\n        pl.plot(self.acc, \'r\')\n        pl.plot(self.loss, \'b\')\n        pl.plot(self.val_acc, \'g\')\n        pl.plot(self.val_loss, \'k\')\n        pl.legend([\'Train acc\',\'Train loss\',\'Valid acc\', \'Valid loss\'], loc=2)\n        display.clear_output(wait=Tr

In [5]:
WEIGHT_DECAY = 0.0005

loss_metric = "categorical_crossentropy"
model.compile(loss=loss_metric, metrics=["accuracy"], 
              optimizer=optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=WEIGHT_DECAY, amsgrad=False))

#histories = Histories()
res = model.fit_generator(train_generator, steps_per_epoch = math.ceil(5075/BATCH_SIZE), epochs=50, verbose=2)


Epoch 1/2000
 - 29s - loss: 2.7734 - acc: 0.2008
Epoch 2/2000
 - 8s - loss: 0.9503 - acc: 0.6337
Epoch 3/2000
 - 8s - loss: 0.4934 - acc: 0.8261
Epoch 4/2000
 - 9s - loss: 0.3507 - acc: 0.8871
Epoch 5/2000
 - 8s - loss: 0.2925 - acc: 0.9098
Epoch 6/2000
 - 8s - loss: 0.2271 - acc: 0.9343
Epoch 7/2000
 - 8s - loss: 0.1894 - acc: 0.9512
Epoch 8/2000
 - 8s - loss: 0.1810 - acc: 0.9496
Epoch 9/2000
 - 8s - loss: 0.1829 - acc: 0.9496
Epoch 10/2000
 - 9s - loss: 0.1528 - acc: 0.9610
Epoch 11/2000
 - 8s - loss: 0.1320 - acc: 0.9633
Epoch 12/2000
 - 8s - loss: 0.0994 - acc: 0.9768
Epoch 13/2000
 - 9s - loss: 0.0874 - acc: 0.9766
Epoch 14/2000
 - 8s - loss: 0.0769 - acc: 0.9795
Epoch 15/2000
 - 8s - loss: 0.0705 - acc: 0.9824
Epoch 16/2000
 - 9s - loss: 0.0696 - acc: 0.9796
Epoch 17/2000
 - 8s - loss: 0.0943 - acc: 0.9696
Epoch 18/2000
 - 8s - loss: 0.0934 - acc: 0.9707
Epoch 19/2000
 - 9s - loss: 0.1004 - acc: 0.9686
Epoch 20/2000
 - 8s - loss: 0.0690 - acc: 0.9784
Epoch 21/2000
 - 8s - loss: 

Epoch 162/2000
 - 8s - loss: 0.0120 - acc: 0.9967
Epoch 163/2000


KeyboardInterrupt: 

In [6]:
scores = model.evaluate(x_train, y_train, verbose=0)
print("Train Accuracy: %.2f%%" % (scores[1]*100))
scores = model.evaluate(x_test, y_test, verbose=0)
print("Test Accuracy: %.2f%%" % (scores[1]*100))

NameError: name 'x_train' is not defined

In [7]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1049061363234677146
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 79508275
locality {
  bus_id: 1
}
incarnation: 10124132242725038463
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:65:00.0, compute capability: 6.1"
]
